In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-vs-real/Data/Test/image_22.jpg
/kaggle/input/ai-vs-real/Data/Test/image_53.jpg
/kaggle/input/ai-vs-real/Data/Test/image_141.jpg
/kaggle/input/ai-vs-real/Data/Test/image_69.jpg
/kaggle/input/ai-vs-real/Data/Test/image_68.jpg
/kaggle/input/ai-vs-real/Data/Test/image_175.jpg
/kaggle/input/ai-vs-real/Data/Test/image_166.jpg
/kaggle/input/ai-vs-real/Data/Test/image_153.jpg
/kaggle/input/ai-vs-real/Data/Test/image_176.jpg
/kaggle/input/ai-vs-real/Data/Test/image_51.jpg
/kaggle/input/ai-vs-real/Data/Test/image_178.jpg
/kaggle/input/ai-vs-real/Data/Test/image_88.jpg
/kaggle/input/ai-vs-real/Data/Test/image_132.jpg
/kaggle/input/ai-vs-real/Data/Test/image_4.jpg
/kaggle/input/ai-vs-real/Data/Test/image_75.jpg
/kaggle/input/ai-vs-real/Data/Test/image_124.jpg
/kaggle/input/ai-vs-real/Data/Test/image_180.jpg
/kaggle/input/ai-vs-real/Data/Test/image_179.jpg
/kaggle/input/ai-vs-real/Data/Test/image_12.jpg
/kaggle/input/ai-vs-real/Data/Test/image_16.jpg
/kaggle/input/ai-vs-real/Data/T

In [19]:
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image,UnidentifiedImageError
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
#HyperParameters:
learning_rate=0.0001
num_epochs=30
batch_size=16
device = torch.device("cuda")
#importing and pre-processing of data
data_dir = '/kaggle/input/ai-vs-real/Data/Train'
real_images_path = os.path.join(data_dir, 'Real')
ai_images_path = os.path.join(data_dir, 'AI')
#seperate real and ai images 
real_images = [os.path.join(real_images_path, f) for f in os.listdir(real_images_path) if f.endswith(('.jpg', '.png'))]
ai_images = [os.path.join(ai_images_path, f) for f in os.listdir(ai_images_path) if f.endswith(('.jpg', '.png'))]
#labelled real images as 0 and ai images as 1
image_paths = real_images + ai_images
labels = [0] * len(real_images) + [1] * len(ai_images) 
#Dividing Training and Validation data

train_paths, valid_paths, train_labels, valid_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

#Creating customDataset
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label


In [20]:
#applying transformations to the image so that it embeds the reality
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),#flips the image 
    transforms.RandomRotation(15),#
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),#add some charecterstics,
    transforms.ToTensor(),
     
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CustomImageDataset(train_paths, train_labels, transform=transform)
valid_dataset = CustomImageDataset(valid_paths, valid_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [21]:
#printing the sample size
example=iter(train_loader)
samples,labels=next(example)
print(samples.shape,labels.shape)

torch.Size([16, 3, 224, 224]) torch.Size([16])


In [22]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [27]:
model=CNN().to(device)
  
loss= nn.CrossEntropyLoss()


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
#scheduler is the optimizer of learnig rate,gamma represents how much the learning rate should be multiplied by 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.8)

for epoch in range(num_epochs):
    model.train()  # Set the model to the  training mode
    epoch_loss = 0.0  # Initialize loss accumulator for the epoch


    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        l = loss(outputs, labels)

        # Backward pass and optimization
        l.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Accumulate loss for epoch
        epoch_loss += l.item()

    # Step the scheduler after the epoch
    scheduler.step()

    # Calculate and print the average epoch loss
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1} Completed. Average Loss: {avg_epoch_loss:.6f}, LR: {scheduler.get_last_lr()[0]:.6f}")
    

Epoch 1 Completed. Average Loss: 0.433012, LR: 0.000100
Epoch 2 Completed. Average Loss: 0.266933, LR: 0.000100
Epoch 3 Completed. Average Loss: 0.186639, LR: 0.000100
Epoch 4 Completed. Average Loss: 0.171726, LR: 0.000100
Epoch 5 Completed. Average Loss: 0.149597, LR: 0.000080
Epoch 6 Completed. Average Loss: 0.143169, LR: 0.000080
Epoch 7 Completed. Average Loss: 0.099164, LR: 0.000080
Epoch 8 Completed. Average Loss: 0.096564, LR: 0.000080
Epoch 9 Completed. Average Loss: 0.125570, LR: 0.000080
Epoch 10 Completed. Average Loss: 0.100362, LR: 0.000064
Epoch 11 Completed. Average Loss: 0.086655, LR: 0.000064
Epoch 12 Completed. Average Loss: 0.060996, LR: 0.000064
Epoch 13 Completed. Average Loss: 0.076500, LR: 0.000064
Epoch 14 Completed. Average Loss: 0.103319, LR: 0.000064
Epoch 15 Completed. Average Loss: 0.063008, LR: 0.000051
Epoch 16 Completed. Average Loss: 0.047678, LR: 0.000051
Epoch 17 Completed. Average Loss: 0.040931, LR: 0.000051
Epoch 18 Completed. Average Loss: 0.0458

In [28]:
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for (images,labels) in (valid_loader):
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    _, predicted_cls = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += (predicted_cls == labels).sum().item()

accuracy=correct/total
print("Accuracy:"+f"{accuracy*100:.3f}")  

Accuracy:98.137


In [29]:
test_images_path = '/kaggle/input/ai-vs-real/Data/Test'


test_images = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.endswith(('.jpg', '.png'))]


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [30]:
#creating custom dataset for loading test images 
class TestImageDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

       
        self.image_paths = [path for path in self.image_paths if self.is_valid_image(path)]

    def is_valid_image(self, img_path):
        try:
         
            Image.open(img_path).convert("RGB")
            return True
        except UnidentifiedImageError:
            print(f"Skipping invalid image: {img_path}")
            return False

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(img_path)

test_dataset = TestImageDataset(test_images, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()  
predictions = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)  
        outputs = model(images)
        _, predicted_cls_test = torch.max(outputs, dim=1)
        for filename, label in zip(filenames, predicted_cls_test):
            
            label_str = 'AI' if label.item() == 1 else 'Real'
            predictions.append((filename, label_str))

Skipping invalid image: /kaggle/input/ai-vs-real/Data/Test/image_62.jpg


In [31]:
import pandas as pd
#submission file of csv
submission = pd.DataFrame(predictions, columns=['Id', 'Label'])



submission.to_csv('submission72.csv', index=False)